In [4]:
"""
FOR GRID INDEXING TASK ONLY
"""


import json
import cv2
import numpy as np
import os
from tqdm import tqdm

with open("/mnt/bn/nlhei-nas/liubangya/proj/vlm/QA/grid_cfg_scannetpp.json", "r") as f:
    cfg = json.load(f)
    H = cfg["H"]
    W = cfg["W"]
    patchsize_H = cfg["patchsize_H"]
    patchsize_W = cfg["patchsize_W"]
# H = 900
# W = 1600
# patchsize_H = 100
# patchsize_W = 100

before_mask = "/mnt/bn/nlhei-nas/liubangya/proj/vlm/workspace/image_hint_16x24_2/pairs/QA_pairs_qwen.train.json"

img_dump = before_mask.replace(".json", "_masked")
if os.path.exists(img_dump):
    # remove the dir
    os.system(f"rm -rf {img_dump}")
os.makedirs(img_dump, exist_ok=True)

with open(before_mask, "r") as f:
    qas = json.load(f)

def coloring(rgb, channel, alpha=0.1):
    if channel == 2:
        new_rgb = rgb * alpha + 255 * (1 - alpha)
        # uint8-fy
        return new_rgb.astype(np.uint8)
    else:
        new_rgb = rgb * alpha
        # uint8-fy
        return new_rgb.astype(np.uint8)


new_qas = []
for qa in tqdm(qas):
    img_name = qa["image"][0]
    post_fix = img_name.split(".")[-1]
    new_img_name = os.path.join(img_dump, f"{qa['id']}.{post_fix}")
    gt = qa["conversations"][-1]["value"]
    gt = gt.replace("```json", "").replace("```", "")
    # gt = json.loads(gt)
    # print(f"gt is {gt}")
    first_rol_col = gt.split(",")[0]
    row = int(first_rol_col.split(" ")[1])
    col = int(first_rol_col.split(" ")[2])
    img = cv2.imread(img_name)
    img = cv2.resize(img, (W, H))
    # coloring the patch into red
    
    # colorint the whole block
    # img[row * patchsize_H : (row+1) * patchsize_H, col * patchsize_W : (col+1) * patchsize_W, 0] = \
    #     coloring(img[row * patchsize_H : (row+1) * patchsize_H, col * patchsize_W : (col+1) * patchsize_W, 0], 0)
    # img[row * patchsize_H : (row+1) * patchsize_H, col * patchsize_W : (col+1) * patchsize_W, 1] = \
    #     coloring(img[row * patchsize_H : (row+1) * patchsize_H, col * patchsize_W : (col+1) * patchsize_W, 1], 1)
    # img[row * patchsize_H : (row+1) * patchsize_H, col * patchsize_W : (col+1) * patchsize_W, 2] = \
    #     coloring(img[row * patchsize_H : (row+1) * patchsize_H, col * patchsize_W : (col+1) * patchsize_W, 2], 2)

    boundary_width = 3
    # channel 0
    img[row * patchsize_H : row * patchsize_H + boundary_width, col * patchsize_W : (col+1) * patchsize_W, 0] = \
        coloring(img[row * patchsize_H : row * patchsize_H + boundary_width, col * patchsize_W : (col+1) * patchsize_W, 0], 0)
    img[(row+1) * patchsize_H - boundary_width: (row+1) * patchsize_H, col * patchsize_W : (col+1) * patchsize_W, 0] = \
        coloring(img[(row+1) * patchsize_H - boundary_width: (row+1) * patchsize_H, col * patchsize_W : (col+1) * patchsize_W, 0], 0)
    img[row * patchsize_H : (row+1) * patchsize_H, col * patchsize_W : col * patchsize_W + boundary_width, 0] = \
        coloring(img[row * patchsize_H : (row+1) * patchsize_H, col * patchsize_W : col * patchsize_W + boundary_width, 0], 0)
    img[row * patchsize_H : (row+1) * patchsize_H, (col+1) * patchsize_W - boundary_width : (col+1) * patchsize_W, 0] = \
        coloring(img[row * patchsize_H : (row+1) * patchsize_H, (col+1) * patchsize_W - boundary_width : (col+1) * patchsize_W, 0], 0)
    # channel 1
    img[row * patchsize_H : row * patchsize_H + boundary_width, col * patchsize_W : (col+1) * patchsize_W, 1] = \
        coloring(img[row * patchsize_H : row * patchsize_H + boundary_width, col * patchsize_W : (col+1) * patchsize_W, 1], 1)
    img[(row+1) * patchsize_H - boundary_width: (row+1) * patchsize_H, col * patchsize_W : (col+1) * patchsize_W, 1] = \
        coloring(img[(row+1) * patchsize_H - boundary_width: (row+1) * patchsize_H, col * patchsize_W : (col+1) * patchsize_W, 1], 1)
    img[row * patchsize_H : (row+1) * patchsize_H, col * patchsize_W : col * patchsize_W + boundary_width, 1] = \
        coloring(img[row * patchsize_H : (row+1) * patchsize_H, col * patchsize_W : col * patchsize_W + boundary_width, 1], 1)
    img[row * patchsize_H : (row+1) * patchsize_H, (col+1) * patchsize_W - boundary_width : (col+1) * patchsize_W, 1] = \
        coloring(img[row * patchsize_H : (row+1) * patchsize_H, (col+1) * patchsize_W - boundary_width : (col+1) * patchsize_W, 1], 1)
    # channel 2
    img[row * patchsize_H : row * patchsize_H + boundary_width, col * patchsize_W : (col+1) * patchsize_W, 2] = \
        coloring(img[row * patchsize_H : row * patchsize_H + boundary_width, col * patchsize_W : (col+1) * patchsize_W, 2], 2)
    img[(row+1) * patchsize_H - boundary_width: (row+1) * patchsize_H, col * patchsize_W : (col+1) * patchsize_W, 2] = \
        coloring(img[(row+1) * patchsize_H - boundary_width: (row+1) * patchsize_H, col * patchsize_W : (col+1) * patchsize_W, 2], 2)
    img[row * patchsize_H : (row+1) * patchsize_H, col * patchsize_W : col * patchsize_W + boundary_width, 2] = \
        coloring(img[row * patchsize_H : (row+1) * patchsize_H, col * patchsize_W : col * patchsize_W + boundary_width, 2], 2)
    img[row * patchsize_H : (row+1) * patchsize_H, (col+1) * patchsize_W - boundary_width : (col+1) * patchsize_W, 2] = \
        coloring(img[row * patchsize_H : (row+1) * patchsize_H, (col+1) * patchsize_W - boundary_width : (col+1) * patchsize_W, 2], 2)

    cv2.imwrite(new_img_name, img)
    new_qa = qa.copy()
    new_qa["image"] = [new_img_name] + qa["image"][1:]
    new_qas.append(new_qa)

after_mask = before_mask.replace(".json", ".masked.json")
with open(after_mask, "w") as f:
    json.dump(new_qas, f, indent=2)
import os
os.system(f"mv {before_mask} {before_mask.replace('.json', '.bk.json')}")
os.system(f"mv {after_mask} {before_mask}")

    

  0%|          | 3/3200 [00:00<01:56, 27.44it/s]

100%|██████████| 3200/3200 [01:50<00:00, 28.89it/s]


0